<a href="https://colab.research.google.com/github/KKawaguchi1219/string-detection/blob/main/string_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from pathlib import Path
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer

# 検出したい語句が書かれたテキストファイル名を入力
# ただし、このテキストファイルは１語句ごとに改行されているものとする
path = 'wordlist.txt'
with open(path, 'r') as f:
  List = f
  wordlist = [i.replace('\n', '') for i in  List.readlines()]
  wordlist = [i.replace('-', ' ') for i in  wordlist]

# 英文が書かれたテキストファイルの名前を入力
file_name = 'test.txt'
string = Path(file_name).read_text().translate(str.maketrans({'.': None, ',': None, '!' : None, '?' : None, '"' : None, '\'' : None, '\n' : ' '}))
words = [i for i in string.split()]

# nltkだと精度が細かすぎて文字数が増えるので、処理を挟む前に文字数をカウント
print(words)
print("{} 文字".format(len(words)))

string = string.translate(str.maketrans({'-' : ' '}))
words = nltk.word_tokenize(string)
pos = nltk.pos_tag(words)

# pandasの練習もかねて
df_pos = pd.DataFrame(pos, columns=['word', 'pos'])

# 原型に変換...精度は不明
lemmatizer =WordNetLemmatizer()
verb = ['VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
adverb = ['RBR', 'RBS', 'RP']
adj = ['JJR', 'JJS']
noun = ['NNS']
for i in range(len(words)):
  if df_pos.at[i, 'pos'] in verb:
    df_pos.at[i, 'word'] = lemmatizer.lemmatize(df_pos.at[i, 'word'], pos="v")
    df_pos.at[i, 'pos'] = 'VB'
  elif df_pos.at[i, 'pos'] in adverb:
    df_pos.at[i, 'word'] = lemmatizer.lemmatize(df_pos.at[i, 'word'], pos="r")
    df_pos.at[i, 'pos'] = 'RB'
  elif df_pos.at[i, 'pos'] in adj:
    df_pos.at[i, 'word'] = lemmatizer.lemmatize(df_pos.at[i, 'word'], pos="a")
    df_pos.at[i, 'pos'] = 'JJ'
  elif df_pos.at[i, 'pos'] in noun:
    df_pos.at[i, 'word'] = lemmatizer.lemmatize(df_pos.at[i, 'word'], pos="n")
    df_pos.at[i, 'pos'] = 'NN'

# パターンマッチング　(ただし、原形に変換したので複数形の名詞を含む熟語はマッチしない)
words = df_pos['word'].to_list()
memo = set()
for i in range(len(wordlist)):
  # 検索するidiomの文字数
  num_of_word = len(wordlist[i].split())

  for j in range(len(words)):
    idiom = [words[k] for k in range(j, j+num_of_word)  if k < len(words)]
    # 検索するidiomの文字数以下は捨てる
    if len(idiom) < num_of_word:
      continue
    # 分割された単語を半角スペースで繋げる
    idiom = ' '.join(idiom)

    if idiom in wordlist:
      memo.add(idiom)

print("使用語句：{}".format(list(memo)))

['stocks', 'and', 'shares', 'investment', 'investor', 'values', 'returned', 'standard', 'of', 'living', 'income', 'fossil-fuel', 'investor', 'greenhouse-gas', 'self-esteem']
15 文字
使用語句：['investor', 'value', 'fuel', 'greenhouse gas', 'return', 'income', 'fossil fuel', 'self esteem', 'standard of living', 'investment']
